In [1]:
################################################################################
# MASTERARBEIT - SKRIPT 05:
# EXPERIMENT 1.4 (Forschungsfrage 1) - AUSREISSERERKENNUNG (TabPFN)
################################################################################
#
# ZWECK DIESES SKRIPTS (Methodik gemäß Abschnitt 3.3.1):
#
# 1. (Laden): Lädt den 'schmutzigen' Rohdatensatz (rfd_main.csv).
#
# 2. (Zielspalten): Fokussiert sich auf dieselben Spalten der 
#    Forschungsfrage 1 ('replies', 'views', 'votes'), um die 
#    methodische Vergleichbarkeit zu gewährleisten.
#
# 3. (Methode): Wendet die vierte Methode (erste moderne KI-Methode) an:
#    TabPFN (Tabular Prior-data Fitted Network), ein vortrainiertes 
#    Transformer-Modell (gemäß Abschnitt 2.2.3.1).
#    Da die Ausreißererkennung eine unüberwachte Aufgabe ist, wird die 
#    'tabpfn-extensions'-Bibliothek ('UnsupervisedModel') verwendet.
#
# 4. (Detektion): Führt das 'OutlierDetectionUnsupervisedExperiment' aus 
#    (aus der 'tabpfn-extensions'-Bibliothek), um die 'outlier_scores' 
#    (Ausreißer-Werte) für jede Zeile zu ermitteln.
#
# 5. (Verarbeitung): Normalisiert die ermittelten 'outlier_scores' auf 
#    einen Bereich von 0 (normal) bis 1 (anomal), um die Ergebnisse 
#    interpretierbar zu machen.
#
# 6. (Speichern): Speichert diese normalisierten 'outlier_scores' (Werte)
#    und die Roh-Scores für die spätere Evaluierung. 
#
################################################################################

# Schritt 1: Notwendige Bibliotheken importieren
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys # Wird für sys.exit() bei kritischen Importfehlern benötigt 
import warnings
import torch # Importieren von PyTorch (technische Grundlage für TabPFN)
import time

# Importieren von Skalierungs-Tools 
from sklearn.preprocessing import MinMaxScaler

# Importieren von TabPFN und den speziellen Erweiterungen
print("Lade notwendige Bibliotheken (Pandas, Numpy, Matplotlib, OS, sys, Torch)...")
try:
    # Die Hauptbibliothek für das vortrainierte Modell
    from tabpfn import TabPFNClassifier, TabPFNRegressor
    # Die Erweiterungsbibliothek für unüberwachte Aufgaben
    from tabpfn_extensions import unsupervised
    # Das Modul, das das 'OutlierDetection'-Experiment enthält
    from tabpfn_extensions.unsupervised import experiments as unueberwachte_experimente
    print("TabPFN und tabpfn-extensions Bibliotheken wurden erfolgreich geladen.")

except ImportError as e:
    print(f"FEHLER: Kritische Bibliotheken (TabPFN/Torch) konnten nicht geladen werden.")
    print(f"Fehlermeldung: {e}")
    print("Stellen Sie sicher, dass die Bibliotheken korrekt installiert sind.")
    sys.exit("Skript gestoppt, da Abhängigkeiten fehlen.")

print("Alle Bibliotheken sind bereit.")
print("=" * 70)
#
################################################################################

# SCHRITT 2: Laden des 'schmutzigen' Rohdatensatzes
print("--- Schritt 2: 'Schmutzigen' Rohdatensatz laden ---")

# Gemäß ZWECK-Schritt 1 wird der 'schmutzige' Rohdatensatz (rfd_main.csv) geladen.
dateipfad = 'rfd_main.csv'
df_schmutzig = pd.read_csv(dateipfad)

# Die .shape-Ausgabe bestätigt die Dimensionen des Datensatzes.
# Dies ist ein Kontrollschritt, um sicherzustellen, dass die korrekte Datei 
# mit allen 1326 Zeilen geladen wurde.
print(f"Datensatz geladen: {dateipfad}")
print(f"Dimensionen (Zeilen, Spalten): {df_schmutzig.shape}")
print("=" * 70)
#
################################################################################

# SCHRITT 3: Zielspalten definieren, begründen und auf NaN prüfen
print("\n--- Schritt 3: Zielspalten definieren, begründen und auf NaN prüfen ---")

# --- BEGRÜNDUNG DER SPALTENAUSWAHL (basierend auf EDA Skript 01, Schritt 4) ---
#
# Für das TabPFN-Modell (eine moderne KI-Methode) werden dieselben 
# Zielspalten wie für die traditionellen (IQR) und klassischen ML-Methoden 
# (LOF, Isolation Forest) verwendet, um eine methodisch konsistente 
# Vergleichbarkeit (gemäß Forschungsfrage 1) zu gewährleisten.
#
# 1. AUSGESCHLOSSENE SPALTE ('Unnamed: 0'): 
#    Diese Spalte wird ausgeschlossen, da sie als irrelevanter, 
#    technischer Index identifiziert wurde.
#
# 2. AUSGEWÄHLTE SPALTEN ('replies', 'views', 'votes'): 
#    Diese drei Spalten wurden als primäre Ziele für die 
#    Ausreißererkennung identifiziert. TabPFN wird 
#    auf diesem 3-dimensionalen Datenraum operieren.
#
# --- ENDE DER BEGRÜNDUNG ---

zielspalten = ['replies', 'views', 'votes']

# Methodische Prüfung auf NaN-Werte (Wissenschaftliche Sorgfalt):
# Wie die meisten ML-Modelle kann TabPFN nicht direkt mit 
# fehlenden Werten (NaN) umgehen. Daher ist diese Überprüfung eine 
# zwingende technische Voraussetzung für die Anwendung der Methode.
print("Prüfung auf fehlende Werte (NaN) in den Zielspalten:")
fehlende_werte = df_schmutzig[zielspalten].isnull().sum()
print(fehlende_werte)

if fehlende_werte.sum() > 0:
    print("\nWARNUNG: Es wurden fehlende Werte gefunden. Diese müssen vor der \
TabPFN-Anwendung behandelt werden.")
else:
    print("\nSTATUS: Keine fehlenden Werte (NaN) in den Zielspalten gefunden. \
Die TabPFN-Methode kann direkt angewendet werden.")

print("=" * 70)
#
################################################################################

# SCHRITT 4: Datenkonvertierung zu PyTorch-Tensoren
print("\n--- Schritt 4: Datenkonvertierung zu PyTorch-Tensoren ---")

# Methodische Anforderung (Technische Grundlage):
# TabPFN basiert auf PyTorch. Daher müssen die Daten aus dem 
# Pandas/Numpy-Format in das PyTorch-eigene 'Tensor'-Format 
# konvertiert werden.

# 1. Extrahieren der Zieldaten (wie in Skript 03 & 04)
X_data = df_schmutzig[zielspalten]
print(f"Daten aus Pandas extrahiert (Shape: {X_data.shape})")

# 2. Konvertierung der Daten (X) in einen 'float' Tensor
#    Der Befehl .float() stellt sicher, dass die Daten als Fließkommazahlen 
#    (z.B. 3.0 statt 3) vorliegen, was für neuronale Netze Standard ist.
try:
    X_tensor = torch.tensor(X_data.values).float()
    print(f"Daten (X) erfolgreich in Tensor konvertiert (Shape: {X_tensor.shape})")
except Exception as e:
    print(f"FEHLER bei der Konvertierung zu Tensor: {e}")
    sys.exit("Skript gestoppt.")

# 3. Definieren der Zielvariable (y)
#    Da dies ein unüberwachtes Experiment zur Ausreißererkennung ist, 
#    haben wir keine Zielvariable (keine "korrekten" Antworten).
#    Daher wird y_tensor, wie im Beispiel gezeigt, als 'None' (Nichts) 
#    definiert.
y_tensor = None
print("Zielvariable (y) als 'None' definiert (unüberwachte Aufgabe).")

# 4. Speichern der Attributnamen
#    Die 'tabpfn-extensions'-Bibliothek benötigt die Spaltennamen 
#    für die interne Verarbeitung (gemäß Beispiel-Code).
attribute_namen = zielspalten
print(f"Attributnamen gespeichert: {attribute_namen}")

print("\nDie Daten sind nun im korrekten Format für das TabPFN-Experiment.")
print("=" * 70)
#
################################################################################

# SCHRITT 5: Initialisierung des unüberwachten TabPFN-Modells
print("\n--- Schritt 5: Initialisierung des unüberwachten TabPFN-Modells ---")

# Methodischer Hintergrund (gemäß ZWECK-Schritt 3):
# Die 'tabpfn-extensions'-Bibliothek erstellt ein unüberwachtes Modell, 
# indem sie intern einen Klassifikator (Classifier) und einen Regressor 
# (Regressor) verwendet.

# Technische Parameter:
#
# 1. n_estimators = 32: Um die Experimente in angemessener Zeit durchzuführen,
#    wird ein Kompromiss gewählt.Diese Entscheidung wird im Protokoll festgehalten.
#
# 2. ignore_pretraining_limits = True (Methodisch ZWINGEND):
#    Das vortrainierte TabPFN-Modell ist standardmäßig auf eine 
#    maximale Anzahl von 1000 Trainingszeilen beschränkt.(Wie durch Tests bewiesen,
#    führt die Standardeinstellung zu einem Fehler).
#    (FEHLER während der Ausführung des Experiments (.run): 
#    Running on CPU with more than 1000 samples is not allowed 
#    by default due to slow performance.
#    To override this behavior, set the environment variable 
#    TABPFN_ALLOW_CPU_LARGE_DATASET=1 or set ignore_pretraining_limits=True.
#    Alternatively, consider using a GPU or the
#    tabpfn-client API: https://github.com/PriorLabs/tabpfn-client
#    Dies kann passieren, wenn die Daten unerwartete Werte enthalten 
#    oder die Systemkonfiguration nicht ausreicht.)
#    Unser Datensatz (df_schmutzig.shape[0]) hat 1326 Zeilen.
#    Dieser Parameter (auf 'True' gesetzt) ist daher zwingend erforderlich,
#    damit TabPFN die Verarbeitung unseres (größeren) Datensatzes 
#    überhaupt zulässt.

try:
    print("Initialisiere TabPFN Classifier und Regressor...")
    # Initialisierung des Klassifikators
    clf = TabPFNClassifier(
        n_estimators=32,
        ignore_pretraining_limits=True
    )
    
    # Initialisierung des Regressors
    reg = TabPFNRegressor(
        n_estimators=32,
        ignore_pretraining_limits=True
    )
    
    # Zusammenbau des unüberwachten Modells 
    # (gemäß der 'tabpfn-extensions' Bibliothek)
    model_unueberwacht = unsupervised.TabPFNUnsupervisedModel(
        tabpfn_clf=clf, tabpfn_reg=reg
    )
    print("Das unüberwachte TabPFN-Modell wurde erfolgreich initialisiert.")

except Exception as e:
    print(f"FEHLER bei der Initialisierung des Modells: {e}")
    print("Stellen Sie sicher, dass die Bibliotheken korrekt installiert sind \
und die Parameter gültig sind.")
    sys.exit("Skript gestoppt.")
print("=" * 70)
#
################################################################################

# SCHRITT 6: Ausführung des TabPFN-Ausreißer-Experiments
print("\n--- Schritt 6: Ausführung des TabPFN-Ausreißer-Experiments ---")

# Gemäß ZWECK-Schritt 4 wird nun das unüberwachte Ausreißer-Experiment
# der 'tabpfn-extensions'-Bibliothek gestartet.

# 1. Initialisierung der Experiment-Klasse
try:
    exp_ausreisser = unueberwachte_experimente.OutlierDetectionUnsupervisedExperiment(
        task_type="unsupervised"
    )
    print("Das Experiment (OutlierDetection) wurde initialisiert.")
except Exception as e:
    print(f"FEHLER bei der Initialisierung der Experiment-Klasse: {e}")
    sys.exit("Skript gestoppt.")

print(f"Starte die Ausführung (.run) auf den Daten (Shape: {X_tensor.shape})...")
print("Dieser Schritt ist rechenintensiv und kann je nach Systemleistung \
(CPU/GPU) einige Minuten dauern.")

# 2. Ausführung des Experiments (.run) MIT LAUFZEITMESSUNG
try:
    start_zeit = time.time()  # Startzeitpunkt messen
    
    ergebnisse = exp_ausreisser.run(
        tabpfn=model_unueberwacht,
        X=X_tensor,
        y=y_tensor,
        attribute_names=attribute_namen,
        should_plot=False 
    )
    
    end_zeit = time.time()    # Endzeitpunkt messen
    laufzeit_sek = end_zeit - start_zeit
    
    print("Das Experiment wurde erfolgreich abgeschlossen.")
    print(f"TabPFN-Laufzeit: {laufzeit_sek:.2f} Sekunden "
          f"(≈ {laufzeit_sek/60:.2f} Minuten).")

except Exception as e:
    print(f"FEHLER während der Ausführung des Experiments (.run): {e}")
    sys.exit("Skript gestoppt.")
print("=" * 70)
#
################################################################################    

# SCHRITT 7: Ergebnisse extrahieren und verarbeiten
print("\n--- Schritt 7: Ergebnisse extrahieren und verarbeiten ---")

# Methodischer Ansatz (gemäß ZWECK-Schritt 4 & 5):
# Die Bibliothek gibt 'outlier_scores' (Werte) zurück,
# nicht 'outlier_indices' (Ja/Nein-Entscheidungen).
# Dies ist ein methodisch wichtiger Unterschied zu LOF/Isolation Forest.

print("Extrahiere 'outlier_scores' aus den Ergebnissen...")
ausreisser_werte = ergebnisse.get('outlier_scores')

if ausreisser_werte is None:
    print("FEHLER: Konnte den Schlüssel 'outlier_scores' nicht in den \
Ergebnissen finden.")
    print(f"Verfügbare Schlüssel sind: {ergebnisse.keys()}")
    sys.exit("Skript gestoppt.")

# Die 'ausreisser_werte' werden in den ursprünglichen DataFrame integriert.
df_ergebnisse_tabpfn = df_schmutzig.copy()
df_ergebnisse_tabpfn['tabpfn_ausreisser_wert'] = ausreisser_werte

print("Die Ausreißer-Werte (Scores) wurden zum DataFrame hinzugefügt.")

# --- Methodische Notwendigkeit: Normalisierung ---
# Um die Scores (Werte) vergleichbar zu machen (z.B. für die Evaluierung),
# normalisieren wir sie auf einen Bereich von 0 bis 1.
# 0 = Geringster Ausreißer-Wert (normal)
# 1 = Höchster Ausreißer-Wert (anomal)

roh_werte = df_ergebnisse_tabpfn[['tabpfn_ausreisser_wert']].values
skalierer = MinMaxScaler()
df_ergebnisse_tabpfn['tabpfn_ausreisser_wert_norm'] = skalierer.fit_transform(roh_werte)

print("Die Ausreißer-Werte wurden auf 0-1 normalisiert ('tabpfn_ausreisser_wert_norm').")

# Zeige die 10 auffälligsten "Signale" als Bestätigung
print("\nDie 10 Zeilen mit den höchsten (normalisierten) Ausreißer-Werten:")
print(df_ergebnisse_tabpfn.sort_values(
    by='tabpfn_ausreisser_wert_norm', ascending=False)
    [zielspalten + ['tabpfn_ausreisser_wert_norm']]
    .head(10)
    .round(4) # für die Lesbarkeit
)
print("=" * 70)
#
################################################################################

# SCHRITT 8: Ergebnisse speichern und Zusammenfassung
print("\n--- Schritt 8: Ergebnisse speichern und Zusammenfassung ---")
print("Dieses Skript (05) ist nun abgeschlossen.")

# Wir stellen sicher, dass der Speicher-Ordner ('ergebnisse') existiert,
# um die Ergebnisse strukturiert abzulegen.
os.makedirs('ergebnisse', exist_ok=True)

# Methodischer Unterschied (ZWINGEND):
# Im Gegensatz zu IQR, LOF und Isolation Forest (die eine Ja/Nein-Liste 
# speicherten), speichern wir hier die Scores (Werte).
# Die Umwandlung dieser Scores in Ja/Nein-Entscheidungen (mittels eines 
# Schwellenwerts) erfolgt erst im finalen Evaluierungs-Skript.

# 1. Definieren des Dateipfads für die Ergebnisse
ergebnis_dateipfad = 'ergebnisse/1.4_tabpfn_ausreisser_scores.csv'

# 2. Speichern des DataFrames (nur relevante Spalten)
# Wir speichern die Original-Indizes (Zeilennummern) und die Scores.
df_ergebnisse_tabpfn.to_csv(
    ergebnis_dateipfad, 
    columns=['tabpfn_ausreisser_wert', 'tabpfn_ausreisser_wert_norm'],
    index=True  # Index (Zeilennummer) wird mitgespeichert
)

print(f"\nErgebnisse (Werte/Scores) wurden in '{ergebnis_dateipfad}' gespeichert.")

# --- ZUSAMMENFASSUNG (Für das Protokoll/die Masterarbeit) ---
# Die Zusammenfassung spiegelt wider, dass wir WERTE (Scores) 
# und nicht ANZAHLEN (Counts) gefunden haben.
print("\n--- ZUSAMMENFASSUNG EXPERIMENT 1.4 (TabPFN) ---")
print(f"Methode:           Moderne KI: TabPFN (unüberwacht)")
print(f"Zieldaten:         {dateipfad} (Shape: {df_schmutzig.shape})")
print(f"Zielspalten:       {zielspalten}")
print(f"Parameter (n_est): {clf.n_estimators}")
print(f"Parameter (ignore):{clf.ignore_pretraining_limits}")
print(f"ERGEBNIS (Art):    Ausreißer-Werte (Scores) für alle 1326 Zeilen wurden \
erzeugt und gespeichert.")
print("=" * 70)
print("Die Evaluierung dieser Scores (Umwandlung in Ja/Nein mittels \
Schwellenwert) erfolgt in einem separaten Evaluierungs-Skript.")
print("=" * 70)
#
################################################################################


Lade notwendige Bibliotheken (Pandas, Numpy, Matplotlib, OS, sys, Torch)...


Invalid input. Please enter `y` or `n`.
TabPFN und tabpfn-extensions Bibliotheken wurden erfolgreich geladen.
Alle Bibliotheken sind bereit.
--- Schritt 2: 'Schmutzigen' Rohdatensatz laden ---
Datensatz geladen: rfd_main.csv
Dimensionen (Zeilen, Spalten): (1326, 15)

--- Schritt 3: Zielspalten definieren, begründen und auf NaN prüfen ---
Prüfung auf fehlende Werte (NaN) in den Zielspalten:
replies    0
views      0
votes      0
dtype: int64

STATUS: Keine fehlenden Werte (NaN) in den Zielspalten gefunden. Die TabPFN-Methode kann direkt angewendet werden.

--- Schritt 4: Datenkonvertierung zu PyTorch-Tensoren ---
Daten aus Pandas extrahiert (Shape: (1326, 3))
Daten (X) erfolgreich in Tensor konvertiert (Shape: torch.Size([1326, 3]))
Zielvariable (y) als 'None' definiert (unüberwachte Aufgabe).
Attributnamen gespeichert: ['replies', 'views', 'votes']

Die Daten sind nun im korrekten Format für das TabPFN-Experiment.

--- Schritt 5: Initialisierung des unüberwachten TabPFN-Modells ---
Ini